In [1]:
#env & clients

PROJECT_ID   = "willy-poc"
BQ_LOCATION  = "US"
DATASET      = "jtcg"
BUCKET       = "willy-poc-crm-agent"

GCS_KNOW = "gs://willy-poc-crm-agent/ai-eng-test-sample-knowledges.csv"
GCS_PROD = "gs://willy-poc-crm-agent/ai-eng-test-sample-products.csv"

import os, io, json, re
import pandas as pd
from urllib.parse import urlparse
from google.cloud import storage, bigquery

os.environ["GCLOUD_PROJECT"] = PROJECT_ID

gcs = storage.Client(project=PROJECT_ID)
bq  = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)

def read_gcs_text(gcs_uri: str) -> str:
    """讀取 GCS 上的文字檔內容（UTF-8）"""
    u = urlparse(gcs_uri)
    return gcs.bucket(u.netloc).blob(u.path.lstrip("/")).download_as_text(encoding="utf-8")


In [2]:
# 設置LLM，模型是gemini-2.5-flash-lite
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

# 和 BigQuery 一樣用同一個 PROJECT_ID，location 用 us-central1
vertexai.init(project=PROJECT_ID, location="us-central1")


GEN_MODEL = GenerativeModel("gemini-2.5-flash-lite")
GEN_CONFIG = GenerationConfig(
    temperature=0.2,        
    max_output_tokens=1024,
)



/opt/conda/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [3]:
#load CSV from GCS, normalize, clear & load to BigQuery

# --- load raw ---
df_k_raw = pd.read_csv(io.StringIO(read_gcs_text(GCS_KNOW)))
df_p_raw = pd.read_csv(io.StringIO(read_gcs_text(GCS_PROD)))

# --- normalize: knowledges ---
kc = {c.lower(): c for c in df_k_raw.columns}
def pick_k(*cands):
    for c in cands:
        if c in kc:
            return df_k_raw[kc[c]]
    return pd.Series([None] * len(df_k_raw))

df_k = pd.DataFrame({
    "id":       pick_k("id"),
    "title":    pick_k("title"),
    "question": pick_k("question", "q", "title"),
    "answer":   pick_k("answer", "body", "content", "text"),
    "url":      pick_k("url", "link", "source"),
    "lang":     pick_k("lang", "language"),
}).astype("string")

df_k["lang"] = df_k["lang"].fillna("zh-TW")

# --- normalize: products ---
pc = {c.lower(): c for c in df_p_raw.columns}
def pick_p(*cands):
    for c in cands:
        if c in pc:
            return df_p_raw[pc[c]]
    return pd.Series([None] * len(df_p_raw))

df_p = pd.DataFrame({
    "sku":         pick_p("sku", "id", "product_sku"),
    "name":        pick_p("name", "title", "product_name"),
    "description": pick_p("desc", "description", "body"),
    "spec":        pick_p("spec", "specs", "attributes"),
    "image_url":   pick_p("image_url", "image", "img"),
    "product_url": pick_p("product_url", "url", "link"),
}).astype("string")

# --- clear & load to BQ (保持原本 API 寫法) ---
for t in ["knowledges", "products"]:
    bq.query(f"DELETE FROM `{PROJECT_ID}.{DATASET}.{t}` WHERE TRUE").result()

bq.load_table_from_dataframe(df_k, f"{PROJECT_ID}.{DATASET}.knowledges").result()
bq.load_table_from_dataframe(df_p, f"{PROJECT_ID}.{DATASET}.products").result()

print("Loaded:", len(df_k), "FAQ rows;", len(df_p), "product rows.")


Loaded: 42 FAQ rows; 6 product rows.


In [4]:
#  create remote embedding model (gemini-embedding-001)

CONNECTION = "us.jtcg"

sql_create_remote_model = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET}.text_embedding_model`
REMOTE WITH CONNECTION `{CONNECTION}`
OPTIONS (
  ENDPOINT = 'gemini-embedding-001'
);
"""
bq.query(sql_create_remote_model).result()
print("Remote embedding model ready (Gemini):", f"{PROJECT_ID}.{DATASET}.text_embedding_model")


Remote embedding model ready (Gemini): willy-poc.jtcg.text_embedding_model


In [5]:
# Cell 4: generate embeddings & update tables

# 1) FAQ: question + answer 一起做 embedding → answer_embedding
bq.query(f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.tmp_k_emb` AS
SELECT
  id,
  ml_generate_embedding_result AS emb
FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{DATASET}.text_embedding_model`,
  (
    SELECT
      id,
      -- 把 question/title + answer 串成同一段文字當作 content
      CONCAT(
        COALESCE(question, title, ''),
        ' ',
        COALESCE(answer, '')
      ) AS content
    FROM `{PROJECT_ID}.{DATASET}.knowledges`
  )
);
""").result()

bq.query(f"""
UPDATE `{PROJECT_ID}.{DATASET}.knowledges` k
SET answer_embedding = t.emb
FROM `{PROJECT_ID}.{DATASET}.tmp_k_emb` t
WHERE k.id = t.id
""").result()

# 2) Products: description → description_embedding（維持原本邏輯）
bq.query(f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.tmp_p_emb` AS
SELECT sku, ml_generate_embedding_result AS emb
FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{DATASET}.text_embedding_model`,
  (
    SELECT
      sku,
      COALESCE(description, name) AS content
    FROM `{PROJECT_ID}.{DATASET}.products`
  )
);
""").result()

bq.query(f"""
UPDATE `{PROJECT_ID}.{DATASET}.products` p
SET description_embedding = t.emb
FROM `{PROJECT_ID}.{DATASET}.tmp_p_emb` t
WHERE p.sku = t.sku
""").result()

print("Embeddings generated & updated.")


Embeddings generated & updated.


In [6]:
# Cell 5: vector search helpers (保留原 API signature)

def search_knowledge(query_text, top_k=5):
    """
    在 knowledges 表中用 answer_embedding 做語意搜尋。
    保留原本參數與名稱，額外多帶回 url 欄位。
    """
    sql = f"""
    SELECT 
        base.id,
        base.title,
        base.answer,
        base.url,
        distance
    FROM VECTOR_SEARCH(
        TABLE `{PROJECT_ID}.{DATASET}.knowledges`,
        'answer_embedding',
        (
            SELECT ml_generate_embedding_result AS query_embedding
            FROM ML.GENERATE_EMBEDDING(
                MODEL `{PROJECT_ID}.{DATASET}.text_embedding_model`,
                (SELECT @query_text AS content)
            )
        ),
        top_k => {top_k},
        options => '{{"use_brute_force": true}}'
    )
    ORDER BY distance
    """
    job_config = bigquery.QueryJobConfig(
        query_parameters=[bigquery.ScalarQueryParameter("query_text", "STRING", query_text)]
    )
    return bq.query(sql, job_config=job_config).result()


def search_products(query_text, top_k=5):
    """
    在 products 表中用 description_embedding 做語意搜尋。
    保留原本參數與名稱，額外多帶回 product_url / image_url。
    """
    sql = f"""
    SELECT 
        base.sku,
        base.name,
        base.description,
        base.product_url,
        base.image_url,
        distance
    FROM VECTOR_SEARCH(
        TABLE `{PROJECT_ID}.{DATASET}.products`,
        'description_embedding',
        (
            SELECT ml_generate_embedding_result AS query_embedding
            FROM ML.GENERATE_EMBEDDING(
                MODEL `{PROJECT_ID}.{DATASET}.text_embedding_model`,
                (SELECT @query_text AS content)
            )
        ),
        top_k => {top_k},
        options => '{{"use_brute_force": true}}'
    )
    ORDER BY distance
    """
    job_config = bigquery.QueryJobConfig(
        query_parameters=[bigquery.ScalarQueryParameter("query_text", "STRING", query_text)]
    )
    return bq.query(sql, job_config=job_config).result()

print("Vector search ready (using brute force for small dataset).")


Vector search ready (using brute force for small dataset).


In [7]:
# Cell 6: orders_db JSON & helper functions

ORDERS_JSON = {
    "brand": "JTCG Shop",
    "generated_at": "2025-08-15T07:00:00Z",
    "orders_db": {
        "u_123456": {
            "orders": [
                {
                    "order_id": "JTCG-202508-10001",
                    "placed_at": "2025-08-10T03:33:00Z",
                    "status": "in_transit",
                    "carrier": "DHL",
                    "tracking": "DHL1234567",
                    "eta": "2025-08-16",
                    "items": [
                        {
                            "sku": "JTCG-ARM-DUAL-PRO-32",
                            "name": "JTCG 雙螢幕氣壓臂 Pro（支援至 32 吋）",
                            "qty": 1
                        },
                        {
                            "sku": "JTCG-CABLE-MGMT-KIT",
                            "name": "JTCG 桌面走線管理組",
                            "qty": 1
                        }
                    ],
                    "shipping_address": "台北市信義區松高路 100 號 10 樓",
                    "contact_phone": "0912-345-678",
                    "order_url": "https://example.com/jtcg/o/JTCG-202508-10001"
                },
                {
                    "order_id": "JTCG-202507-09012",
                    "placed_at": "2025-07-22T08:15:00Z",
                    "status": "delivered",
                    "carrier": "T-EX",
                    "tracking": "TEX998877",
                    "eta": "2025-07-26",
                    "items": [
                        {
                            "sku": "JTCG-ARM-SINGLE-LITE-27",
                            "name": "JTCG 單臂支架 Lite（至 27 吋）",
                            "qty": 1
                        }
                    ],
                    "shipping_address": "新北市板橋區文化路一段 200 號 7 樓",
                    "contact_phone": "0922-000-111",
                    "order_url": "https://example.com/jtcg/o/JTCG-202507-09012"
                }
            ]
        },
        "u_999999": {
            "orders": [
                {
                    "order_id": "JTCG-202508-11021",
                    "placed_at": "2025-08-12T10:02:00Z",
                    "status": "processing",
                    "carrier": None,
                    "tracking": None,
                    "eta": None,
                    "items": [
                        {
                            "sku": "JTCG-ARM-ULTRAWIDE-49",
                            "name": "JTCG 超寬螢幕重載臂（至 49 吋）",
                            "qty": 1
                        }
                    ],
                    "shipping_address": "高雄市鼓山區明誠三路 88 號 3 樓",
                    "contact_phone": "0933-222-333",
                    "order_url": "https://example.com/jtcg/o/JTCG-202508-11021"
                }
            ]
        },
        "u_empty": {
            "orders": []
        },
        "u_222222": {
            "orders": [
                {
                    "order_id": "JTCG-202508-12005",
                    "placed_at": "2025-08-12T14:20:00Z",
                    "status": "shipped",
                    "carrier": "T-EX",
                    "tracking": "TEX112233",
                    "eta": "2025-08-17",
                    "items": [
                        {
                            "sku": "JTCG-ARM-ULTRAWIDE-49",
                            "name": "JTCG 超寬螢幕重載臂（至 49 吋）",
                            "qty": 1
                        }
                    ],
                    "shipping_address": "桃園市中壢區中大路 59 號 5 樓",
                    "contact_phone": "0918-111-222",
                    "order_url": "https://example.com/jtcg/o/JTCG-202508-12005"
                },
                {
                    "order_id": "JTCG-202506-05077",
                    "placed_at": "2025-06-05T03:10:00Z",
                    "status": "delivered",
                    "carrier": "DHL",
                    "tracking": "DHL556677",
                    "eta": "2025-06-10",
                    "items": [
                        {
                            "sku": "JTCG-ARM-SINGLE-LITE-27",
                            "name": "JTCG 單臂支架 Lite（至 27 吋）",
                            "qty": 1
                        },
                        {
                            "sku": "JTCG-CABLE-MGMT-KIT",
                            "name": "JTCG 桌面走線管理組",
                            "qty": 1
                        }
                    ],
                    "shipping_address": "桃園市平鎮區新富三街 88 號 9 樓",
                    "contact_phone": "0909-333-444",
                    "order_url": "https://example.com/jtcg/o/JTCG-202506-05077"
                }
            ]
        },
        "u_333333": {
            "orders": [
                {
                    "order_id": "JTCG-202508-14011",
                    "placed_at": "2025-08-14T09:45:00Z",
                    "status": "processing",
                    "carrier": None,
                    "tracking": None,
                    "eta": None,
                    "items": [
                        {
                            "sku": "JTCG-ARM-DUAL-PRO-32",
                            "name": "JTCG 雙螢幕氣壓臂 Pro（支援至 32 吋）",
                            "qty": 1
                        }
                    ],
                    "shipping_address": "台中市西屯區文心路三段 300 號 12 樓",
                            "contact_phone": "0966-000-555",
                            "order_url": "https://example.com/jtcg/o/JTCG-202508-14011"
                },
                {
                    "order_id": "JTCG-202507-30021",
                    "placed_at": "2025-07-30T12:05:00Z",
                    "status": "delivered",
                    "carrier": "T-EX",
                    "tracking": "TEX445566",
                    "eta": "2025-08-03",
                    "items": [
                        {
                            "sku": "JTCG-WALL-ARM-34",
                            "name": "JTCG 壁掛式伸縮臂（至 34 吋）",
                            "qty": 1
                        }
                    ],
                    "shipping_address": "台中市北屯區松竹路二段 120 號 6 樓",
                    "contact_phone": "0955-121-212",
                    "order_url": "https://example.com/jtcg/o/JTCG-202507-30021"
                }
            ]
        },
        "u_444444": {
            "orders": [
                {
                    "order_id": "JTCG-202508-09031",
                    "placed_at": "2025-08-09T07:22:00Z",
                    "status": "in_transit",
                    "carrier": "BlackCat",
                    "tracking": "BC123456789",
                    "eta": "2025-08-15",
                    "items": [
                        {
                            "sku": "JTCG-ARM-SINGLE-LITE-27",
                            "name": "JTCG 單臂支架 Lite（至 27 吋）",
                            "qty": 1
                        },
                        {
                            "sku": "JTCG-LAPTOP-VESA-KIT",
                            "name": "JTCG 筆電托盤＋VESA 轉接組",
                            "qty": 1
                        }
                    ],
                    "shipping_address": "新竹市東區光復路二段 101 號 4 樓",
                    "contact_phone": "0932-777-888",
                    "order_url": "https://example.com/jtcg/o/JTCG-202508-09031"
                }
            ]
        },
        "u_555555": {
            "orders": [
                {
                    "order_id": "JTCG-202508-01041",
                    "placed_at": "2025-08-01T11:33:00Z",
                    "status": "delivered",
                    "carrier": "DHL",
                    "tracking": "DHL888999",
                    "eta": "2025-08-06",
                    "items": [
                        {
                            "sku": "JTCG-ARM-DUAL-PRO-32",
                            "name": "JTCG 雙螢幕氣壓臂 Pro（支援至 32 吋）",
                            "qty": 1
                        }
                    ],
                    "shipping_address": "台南市東區崇學路 66 號 10 樓",
                    "contact_phone": "0977-333-222",
                    "order_url": "https://example.com/jtcg/o/JTCG-202508-01041"
                },
                {
                    "order_id": "JTCG-202507-28052",
                    "placed_at": "2025-07-28T15:18:00Z",
                    "status": "processing",
                    "carrier": None,
                    "tracking": None,
                    "eta": None,
                    "items": [
                        {
                            "sku": "JTCG-CABLE-MGMT-KIT",
                            "name": "JTCG 桌面走線管理組",
                            "qty": 2
                        }
                    ],
                    "shipping_address": "台南市北區成功路 200 號 3 樓",
                    "contact_phone": "0988-654-321",
                    "order_url": "https://example.com/jtcg/o/JTCG-202507-28052"
                }
            ]
        },
        "u_777777": {
            "orders": [
                {
                    "order_id": "JTCG-202505-21073",
                    "placed_at": "2025-05-21T02:20:00Z",
                    "status": "delivered",
                    "carrier": "T-EX",
                    "tracking": "TEX223344",
                    "eta": "2025-05-26",
                    "items": [
                        {
                            "sku": "JTCG-WALL-ARM-34",
                            "name": "JTCG 壁掛式伸縮臂（至 34 吋）",
                            "qty": 1
                        }
                    ],
                    "shipping_address": "高雄市左營區自由一路 150 號 8 樓",
                    "contact_phone": "0910-888-000",
                    "order_url": "https://example.com/jtcg/o/JTCG-202505-21073"
                },
                {
                    "order_id": "JTCG-202508-15083",
                    "placed_at": "2025-08-15T04:12:00Z",
                    "status": "processing",
                    "carrier": None,
                    "tracking": None,
                    "eta": None,
                    "items": [
                        {
                            "sku": "JTCG-ARM-ULTRAWIDE-49",
                            "name": "JTCG 超寬螢幕重載臂（至 49 吋）",
                            "qty": 1
                        }
                    ],
                    "shipping_address": "高雄市鼓山區青海路 88 號 15 樓",
                    "contact_phone": "0935-101-202",
                    "order_url": "https://example.com/jtcg/o/JTCG-202508-15083"
                }
            ]
        }
    }
}

ORDERS_DB = ORDERS_JSON["orders_db"]

def get_orders_by_user(user_id: str):
    rec = ORDERS_DB.get(user_id)
    if not rec:
        return []
    return rec.get("orders", [])

def get_order_detail(user_id: str, order_id: str):
    for o in get_orders_by_user(user_id):
        if o.get("order_id") == order_id:
            return o
    return None


In [8]:
# Cell 7: handover_simple mock (原題目程式碼，直接貼入)

import re as _re_handover

EMAIL_RE = _re_handover.compile(r"^[^\s@]+@[^\s@]+\.[^\s@]+$")

def handover_simple(conversation_id: str, email: str, summary: str, simulate_fail: bool = False) -> str:
    """
    簡易轉接真人（模擬版）
    Args:
        conversation_id: 模擬的對話 ID
        email: 使用者 Email（此版僅做基本格式檢查）
        summary: 對話摘要（會送入 mock API；此處不做實際保存）
        simulate_fail: 設為 True 可強制模擬 API 失敗
    Returns:
        str: 成功 -> "已為您轉接真人"
             失敗 -> "轉接真人時發生錯誤，請聯繫技術團隊協助"
    """
    # 基本 Email 檢查（不通過就視為失敗）
    if not EMAIL_RE.match(email or ""):
        return "轉接真人時發生錯誤，請聯繫技術團隊協助"

    payload = {
        "conversation_id": str(conversation_id),
        "email": email,
        "summary": (summary or "")[:500],  # 簡單截斷，避免過長
    }

    try:
        ok = _mock_api_call(payload, simulate_fail=simulate_fail)
        return "已為您轉接真人" if ok else "轉接真人時發生錯誤，請聯繫技術團隊協助"
    except Exception:
        return "轉接真人時發生錯誤，請聯繫技術團隊協助"

def _mock_api_call(payload: dict, simulate_fail: bool = False) -> bool:
    """
    假的 API 呼叫：預設成功。
    - simulate_fail=True 時強制回傳失敗
    - 或者當 conversation_id 以 "FAIL" 開頭時回傳失敗（方便寫測試）
    """
    if simulate_fail:
        return False
    if payload.get("conversation_id", "").upper().startswith("FAIL"):
        return False
    return True


In [9]:
# Cell 8: intent detection rule (保留函式名稱與參數)

def detect_intent_rule(query: str) -> str:
    """
    簡易規則式 intent 分類。
    保留原 API：輸入 query 字串，輸出 "FAQ" / "ORDER" / "HANDOFF" / "PRODUCT"。
    """
    q = (query or "").lower()

    # HANDOFF 優先：真人客服
    if any(kw in q for kw in ["真人", "客服", "轉接", "人工", "human agent", "talk to a person", "talk to agent"]):
        return "HANDOFF"

    # ORDER：帶有「訂單編號 / order id / 我的訂單」等語意才進入
    if ("訂單" in q and any(kw in q for kw in ["查", "看", "幫我", "狀態", "這筆"])) \
       or "order id" in q \
       or "order_id" in q \
       or "jtcg-" in q:
        return "ORDER"

    # PRODUCT：產品 / 推薦 / 規格 / 支架 / 螢幕 臂 等
    if any(kw in q for kw in [
        "產品", "推薦", "哪支臂", "螢幕臂", "支架", "壁掛", "走線", "vesa",
        "product", "recommend", "spec", "which arm", "mount"
    ]):
        return "PRODUCT"

    # 其他預設當 FAQ
    return "FAQ"


In [11]:
def generate_llm_reply(user_query: str, context_blocks: list[str], lang: str = "zh-TW") -> str:
    """
    用 Gemini 2.5 Pro 把「使用者問題 + 檢索回來的 FAQ / 商品內容」包成最終回答。
    - 只走中文路徑，lang 參數僅保留介面相容，不做分支。
    - context_blocks[0] 視為與問題最相近的一筆，標記為 [最高相關資料]。
    """
    if not context_blocks:
        context_text = "（目前沒有檢索到任何 FAQ 或商品內容。）"
    else:
        main_block = context_blocks[0]
        other_blocks = context_blocks[1:]

        parts = [f"[最高相關資料]\n{main_block}"]
        for i, block in enumerate(other_blocks, start=1):
            parts.append(f"[參考資料 {i}]\n{block}")
        context_text = "\n\n".join(parts)

    prompt = f"""
你是 JTCG Shop 的官方客服 AI，請用自然、有禮貌的繁體中文回答。

【使用者問題】
{user_query}

【系統檢索到的相關資料】
{context_text}

接下來請依照以下規則回答：

1. 請將標記為「[最高相關資料]」的內容視為目前系統判斷與問題「最相近的一筆資料」：
   - 你應該優先只依據這一筆資料回答。
   - 如果這一筆資料已經提供了明確、具體的答案（例如：
     ‧ 保固期間（如「享有 1 年有限保固」）
     ‧ 是否可以開立三聯式發票、如何填寫統一編號
     ‧ 出貨大約需要幾天、預估到貨時間
     ‧ 是否有免運、運費如何計算
     ‧ 退貨 / 換貨 / 退款規則
     ‧ 優惠券可以如何使用、有沒有併用限制
     等），請直接整理並引用「[最高相關資料]」作為最終回答。
   - 在這種情況下，請忽略所有「[參考資料]」，不要混合或綜合多筆資料，也不要說「資料不足」或「無法確認」。

2. 只有在「[最高相關資料]」明顯與問題主題無關，或內容完全無法回答問題時，你才可以：
   - 參考一到兩筆「[參考資料]」作為補充說明，或
   - 判斷所有資料都不足以提供可靠的答案。

3. 若包含「[最高相關資料]」在內的所有資料都無法提供任何可靠線索，才可以回答：
   「目前無法根據現有資料做出可靠判斷」，並建議使用者改由真人客服協助。

4. 對於保固、退換貨、出貨時間、運費、優惠券與活動、發票與統編等「政策與條款」：
   - 你只能依照以上檢索到的內容作答；
   - 不可以自行臆測、延伸或發明新的規則。

5. 如果資料中有出現網址（URL），請在回覆中附上最相關的一到兩個連結，
   讓使用者可以直接點擊查看詳細說明。

6. 回覆時請：
   - 先用一到兩句話清楚說出「主要答案」；
   - 若有需要，再簡短補充重要注意事項或下一步建議；
   - 語氣保持專業、可信、友善，避免過度冗長。

最後，請只輸出要給使用者看的「最終回覆內容」：
- 不要逐條重複貼出上述資料的原文，
- 也不要解釋你的思考過程或使用到哪些規則。
"""

    try:
        response = GEN_MODEL.generate_content(
            prompt,
            generation_config=GEN_CONFIG,
        )
        text = (response.text or "").strip()
        if text:
            return text
    except Exception as e:
        print("LLM error in generate_llm_reply:", e)

    # fallback：LLM 出錯時的保守回應
    return "抱歉，產生回答時發生問題，建議您稍後再試，或直接改由真人客服協助。"


In [12]:
# Cell 9: handlers & answer_with_rag (保留原函式名稱與參數)

URL_IN_TEXT_RE = re.compile(r"https?://\S+")
USER_ID_RE     = re.compile(r"\bu_\d{6}\b", re.IGNORECASE)
ORDER_ID_RE    = re.compile(r"\bJTCG-\d{6}-\d{5}\b", re.IGNORECASE)

def _detect_lang(text: str, default: str = "zh-TW") -> str:
    """非常簡單的語言偵測：英文字母佔比高就當英文，否則用 default。"""
    if not text:
        return default
    letters = sum(ch.isalpha() for ch in text)
    ascii_letters = sum(ch.isascii() and ch.isalpha() for ch in text)
    if letters > 0 and ascii_letters / max(letters, 1) > 0.7:
        return "en"
    return default

# 常見 FAQ 主題關鍵字（可以慢慢補）
FAQ_KEYWORDS = [
    "保固", "維修", "RMA",
    "發票", "三聯", "統編",
    "出貨", "到貨", "運費", "免運",
    "退貨", "換貨", "退款",
    "優惠券", "折扣", "活動",
]

def _keyword_overlap_score(query: str, text: str) -> int:
    """
    回傳 query 與 text 之間的關鍵字重疊數量。
    - 每一個 FAQ_KEYWORDS，若同時存在於 query 與 text，就 +1。
    """
    score = 0
    for kw in FAQ_KEYWORDS:
        if kw in query and kw in text:
            score += 1
    return score

def handle_faq(query: str, top_k: int = 5, lang: str = "zh-TW") -> str:
    """
    FAQ 情境：
    1. 先用 BigQuery VECTOR_SEARCH 找出 top_k 筆 FAQ
    2. 在 Python 端加一層 keyword-aware re-ranking
    3. 高信心命中就直接用 FAQ 回覆，否則再交給 LLM 統整
    """
    rows = list(search_knowledge(query, top_k=top_k))

    # 完全找不到 FAQ → 丟給 LLM，讓它走「沒有資料」的保守路徑
    if not rows:
        return generate_llm_reply(query, [], lang=lang)

    # 先做 keyword-aware re-ranking
    candidates = []
    for r in rows:
        title    = (r.title  or "").strip()
        answer   = (r.answer or "").strip()
        text_all = title + " " + answer
        dist     = getattr(r, "distance", None)
        kw_score = _keyword_overlap_score(query, text_all)

        candidates.append({
            "row": r,
            "kw_score": kw_score,
            "distance": dist if dist is not None else 9999.0,
        })

    # 排序規則：
    # 1) kw_score 由大到小（關鍵字越對得上的越前面）
    # 2) distance 由小到大（向量越近越前面）
    candidates.sort(key=lambda x: (-x["kw_score"], x["distance"]))

    # 重新排好 rows（照新的 ranking 順序）
    rows_sorted = [c["row"] for c in candidates]
    best        = rows_sorted[0]
    best_dist   = getattr(best, "distance", None)
    best_title  = (best.title  or "").strip()
    best_answer = (best.answer or "").strip()
    best_url    = (getattr(best, "url", None) or "").strip()

    # 再算一次「最優那筆」的 keyword match（強信心直答用）
    text_for_match   = best_title + best_answer
    best_kw_overlap  = _keyword_overlap_score(query, text_for_match)

    # 規則：如果關鍵字有對上，且向量距離不是離超遠，就直接用 FAQ 回覆
    if best_answer and best_kw_overlap > 0 and (best_dist is None or best_dist < 0.9):
        if lang.startswith("en"):
            resp = "Here is what I found in our FAQ:\n\n"
            resp += best_answer
            if best_url:
                resp += f"\n\nYou can find more details here: {best_url}"
            return resp.strip()

        # 中文：直答 FAQ，再補充連結
        parts = [best_answer]
        if best_url:
            parts.append(f"\n\n更多詳細說明，您可以參考：{best_url}")
        return "".join(parts)

    # 若沒有明顯高信心命中，就把多筆 FAQ 丟給 LLM 統整
    context_blocks = []
    for r in rows_sorted:
        t_title    = (r.title  or "").strip()
        t_answer   = (r.answer or "").strip()
        t_url      = (getattr(r, "url", None) or "").strip()
        t_distance = getattr(r, "distance", None)

        block_lines = []
        if t_title:
            block_lines.append(f"FAQ 標題：{t_title}")
        if t_answer:
            block_lines.append(f"FAQ 內容：{t_answer}")
        if t_url:
            block_lines.append(f"相關連結：{t_url}")
        if t_distance is not None:
            block_lines.append(f"語意距離（數值越小越相近）：{t_distance}")

        context_blocks.append("\n".join(block_lines))

    return generate_llm_reply(query, context_blocks, lang=lang)


def handle_product(query: str, top_k: int = 3, lang: str = "zh-TW") -> str:
    rows = list(search_products(query, top_k=top_k))
    if not rows:
        if lang.startswith("en"):
            return "I couldn't find matching products. Please tell me your monitor size, weight, and how you plan to mount it, so I can narrow down the options."
        return "目前沒有找到明確符合的商品。您可以告訴我螢幕尺寸、重量以及預計安裝方式（桌夾或壁掛），我可以再幫您縮小選項。"

    if lang.startswith("en"):
        header = "Here are some products that may fit your needs:\n\n"
    else:
        header = "根據您的需求，以下是幾款可能適合的商品：\n\n"

    lines = [header]
    for idx, r in enumerate(rows[:3], start=1):
        name = (r.name or "").strip()
        sku  = (r.sku or "").strip()
        desc = (r.description or "").strip()
        url  = (getattr(r, "product_url", None) or "").strip()
        img  = (getattr(r, "image_url", None) or "").strip()

        if lang.startswith("en"):
            line = f"{idx}. {name} (SKU: {sku})\n{desc}\n"
            if url:
                line += f"Product page: {url}\n"
            if img:
                line += f"Image: {img}\n"
        else:
            line = f"{idx}. 【{name}】（SKU：{sku}）\n{desc}\n"
            if url:
                line += f"商品連結：{url}\n"
            if img:
                line += f"商品圖片：{img}\n"
        lines.append(line + "\n")

    if lang.startswith("en"):
        lines.append("If you share your monitor size and weight, I can further narrow down to the most suitable option.")
    else:
        lines.append("如果您願意，也可以跟我說螢幕尺寸與重量，我可以再幫您挑出最適合的一款。")

    return "".join(lines).strip()


def handle_order(query: str, lang: str = "zh-TW") -> str:
    # 嘗試從文字中抓 user_id 與 order_id
    user_match  = USER_ID_RE.search(query)
    order_match = ORDER_ID_RE.search(query)

    user_id  = user_match.group(0) if user_match else None
    order_id = order_match.group(0) if order_match else None

    if not user_id and not order_id:
        # 沒有任何 ID → 第一階段：要求 user_id（維持原有行為）
        if lang.startswith("en"):
            return "To look up an order, please first provide your user_id (for example: u_123456)."
        return "要查訂單請先提供 user_id（例如：u_123456）。回覆 user_id 後我會列出可選的 order_id。"

    if user_id and not order_id:
        # 有 user_id，列出訂單清單
        orders = get_orders_by_user(user_id)
        if not orders:
            if lang.startswith("en"):
                return f"I couldn't find any orders for user_id {user_id}. Please confirm your user_id or contact a human agent."
            return f"目前找不到 user_id「{user_id}」對應的訂單紀錄，請您確認帳號是否正確，或改由真人客服協助。"

        if lang.startswith("en"):
            header = f"Here are the orders under user_id {user_id}:\n\n"
        else:
            header = f"以下是 user_id「{user_id}」名下的訂單：\n\n"

        lines = [header]
        for o in orders:
            oid  = o.get("order_id")
            stat = o.get("status")
            eta  = o.get("eta")
            items = ", ".join([it.get("name") for it in o.get("items", [])])
            if lang.startswith("en"):
                line = f"- {oid}: status = {stat}, eta = {eta}, items = {items}"
            else:
                line = f"- 訂單編號 {oid}：狀態 {stat}，預估到貨 {eta}，品項：{items}"
            lines.append(line)

        if lang.startswith("en"):
            lines.append("\nPlease reply with the order_id you want to check in detail.")
        else:
            lines.append("\n請回覆想要查詢的訂單編號（order_id），我可以再提供更詳盡的物流與品項資訊。")

        return "\n".join(lines)

    if user_id and order_id:
        # 同時有 user_id + order_id → 回傳詳細資訊
        order = get_order_detail(user_id, order_id)
        if not order:
            if lang.startswith("en"):
                return f"I couldn't find order {order_id} under user_id {user_id}. Please double-check the information or contact a human agent."
            return f"在 user_id「{user_id}」底下找不到訂單編號 {order_id}，請確認資訊是否正確，或改由真人客服協助。"

        stat   = order.get("status")
        carrier = order.get("carrier") or "尚未建立"
        tracking = order.get("tracking") or "尚未建立"
        eta    = order.get("eta") or "尚未提供"
        items  = order.get("items", [])
        addr   = order.get("shipping_address")
        url    = order.get("order_url")

        if lang.startswith("en"):
            lines = [
                f"Order {order_id} status: {stat}",
                f"- Carrier: {carrier}",
                f"- Tracking: {tracking}",
                f"- ETA: {eta}",
                f"- Shipping address: {addr}",
                "- Items:"
            ]
            for it in items:
                lines.append(f"  * {it.get('name')} x{it.get('qty')}")
            if url:
                lines.append(f"\nYou can also view the order here: {url}")
            return "\n".join(lines)

        lines = [
            f"訂單 {order_id} 目前狀態：{stat}",
            f"- 配送業者：{carrier}",
            f"- 追蹤碼：{tracking}",
            f"- 預估到貨日：{eta}",
            f"- 寄送地址：{addr}",
            "- 購買品項："
        ]
        for it in items:
            lines.append(f"  ‧ {it.get('name')} x{it.get('qty')}")

        if url:
            lines.append(f"\n您也可以直接透過這個連結查看訂單詳情：{url}")

        return "\n".join(lines)

    # 若只有 order_id 沒有 user_id，就不要洩漏資訊
    if order_id and not user_id:
        if lang.startswith("en"):
            return "For security reasons, I need your user_id (e.g., u_123456) to look up this order."
        return "基於帳號安全考量，查詢訂單時需要同時提供 user_id（例如：u_123456）。請先提供 user_id，我再幫您查詢。"


def handle_handoff(query: str, lang: str = "zh-TW") -> str:
    # 嘗試從文字中抓 email
    email_match = EMAIL_RE.search(query)
    if not email_match:
        # 保持原本第一句話術
        if lang.startswith("en"):
            return "Understood. Please provide your email address so we can verify it and transfer you to a human agent."
        return "已了解你需要真人協助，請提供 Email（我們會先驗證格式再轉接）。"

    email = email_match.group(0)
    # 這裡用一個固定 conversation_id，summary 就用 query 本身
    msg = handover_simple("JTCG-CHAT-BATCH", email, summary=query)
    return msg


def answer_with_rag(query_text: str, top_k: int = 3, lang_hint: str = "zh-TW") -> str:
    """
    主入口：保留原有 API（query_text, top_k, lang_hint）。
    依照 intent 分派到 FAQ / PRODUCT / ORDER / HANDOFF handler。
    """
    query_text = (query_text or "").strip()
    if not query_text:
        return ""

    lang   = _detect_lang(query_text, default=lang_hint)
    intent = detect_intent_rule(query_text)

    if intent == "ORDER":
        return handle_order(query_text, lang=lang)
    elif intent == "HANDOFF":
        return handle_handoff(query_text, lang=lang)
    elif intent == "PRODUCT":
        return handle_product(query_text, top_k=top_k, lang=lang)
    else:
        # FAQ + out-of-scope fallback（現在會走 LLM）
        resp = handle_faq(query_text, top_k=top_k, lang=lang)
        return resp


In [13]:
# === New Cell: 單點測試 FAQ / PRODUCT / ORDER / HANDOFF ===

tests = [
    "保固多久？維修怎麼申請？",
    "發票可以開三聯式嗎？要怎麼填統編？",
    "出貨大概幾天？有免運嗎？",
    "VESA 100x100 的螢幕可以用哪支臂？",
    "幫我查一下訂單出貨了沒",
    "我要找真人客服"
]

for q in tests:
    print("Q:", q)
    print("A:", answer_with_rag(q, top_k=10, lang_hint="zh-TW"))
    print("-" * 100)


Q: 保固多久？維修怎麼申請？
A: 請備妥訂單編號與故障影片/照片，於線上表單提交 RMA。客服會於工作日內回覆檢測與處理方式。
----------------------------------------------------------------------------------------------------
Q: 發票可以開三聯式嗎？要怎麼填統編？
A: 本店提供電子發票，出貨完成後將以 Email 寄送或於會員中心可下載查詢。若需三聯式發票，請於下單時備註統編與抬頭。
----------------------------------------------------------------------------------------------------
Q: 出貨大概幾天？有免運嗎？
A: 現貨商品於 1–2 個工作天出貨；預購商品依頁面標示為準。標準運送滿 NT$1,500 免運；離島或外島運費另計。
----------------------------------------------------------------------------------------------------
Q: VESA 100x100 的螢幕可以用哪支臂？
A: 根據您的需求，以下是幾款可能適合的商品：

1. 【JTCG 壁掛式伸縮臂（至 34 吋）】（SKU：JTCG-WALL-ARM-34）

商品連結：https://example.com/jtcg/p/JTCG-WALL-ARM-34

2. 【JTCG 超寬螢幕重載臂（至 49 吋）】（SKU：JTCG-ARM-ULTRAWIDE-49）

商品連結：https://example.com/jtcg/p/JTCG-ARM-ULTRAWIDE-49

3. 【JTCG 單臂支架 Lite（至 27 吋）】（SKU：JTCG-ARM-SINGLE-LITE-27）

商品連結：https://example.com/jtcg/p/JTCG-ARM-SINGLE-LITE-27

如果您願意，也可以跟我說螢幕尺寸與重量，我可以再幫您挑出最適合的一款。
-------------------------------------------------------------

In [14]:
# Cell 11: batch run on sample conversations & export CSV

from tqdm import tqdm  # 進度條

# 1) GCS 路徑
GCS_CONV = "gs://willy-poc-crm-agent/ai-eng-test-sample-conversations.json"

def read_gcs_json(gcs_uri: str):
    """從 GCS 讀取 JSON 檔案"""
    try:
        u = urlparse(gcs_uri)
        blob = gcs.bucket(u.netloc).blob(u.path.lstrip("/"))
        content = blob.download_as_text()
        return json.loads(content)
    except Exception as e:
        print(f"❌ 讀取 GCS 失敗: {gcs_uri}")
        print(f"   錯誤: {e}")
        return []

print(f"📥 讀取對話資料: {GCS_CONV}")
dialogs = read_gcs_json(GCS_CONV)
print(f"✅ 載入 {len(dialogs)} 組對話")

rows = []

for i, dlg in enumerate(tqdm(dialogs, desc="🤖 處理對話"), start=1):
    # 提取最後一個使用者訊息
    last_user = ""
    for t in dlg:
        if t.get("role") == "user":
            if "content" in t and isinstance(t["content"], list):
                parts = [
                    p.get("text", "")
                    for p in t["content"]
                    if isinstance(p, dict) and p.get("type") == "text"
                ]
                txt = " ".join(parts).strip()
            else:
                txt = (t.get("text") or "").strip()
            if txt:
                last_user = txt

    if not last_user:
        rows.append({
            "idx": i,
            "user_query": "",
            "detected_intent": "",
            "agent_response": "⚠️ 無使用者訊息",
            "source_links": "",
            "is_in_scope_and_correct": ""
        })
        continue

    intent = detect_intent_rule(last_user)

    try:
        resp = answer_with_rag(last_user, top_k=3, lang_hint="zh-TW")
        links = " ".join(URL_IN_TEXT_RE.findall(resp or ""))
    except Exception as e:
        resp = f"❌ 處理錯誤: {str(e)[:100]}"
        links = ""
        print(f"⚠️  對話 {i} 處理失敗: {str(e)[:50]}")

    rows.append({
        "idx": i,
        "user_query": last_user,
        "detected_intent": intent,
        "agent_response": resp,
        "source_links": links,
        "is_in_scope_and_correct": ""  # 保留給人工標註
    })

df_eval = pd.DataFrame(rows)

# 儲存到本地
import os as _os
out_dir = _os.path.expanduser("~/jtcg_outputs")
_os.makedirs(out_dir, exist_ok=True)
local_csv = _os.path.join(out_dir, "jtcg_agent_runs.csv")
df_eval.to_csv(local_csv, index=False, encoding="utf-8-sig")
print(f"✅ 已儲存: {local_csv}")

# 統計資訊
print("\n📊 Intent 分布:")
print(df_eval["detected_intent"].value_counts())

print(f"\n📈 總共處理: {len(df_eval)} 組對話")
print(f"   有效查詢: {(df_eval['user_query'] != '').sum()} 組")
print(f"   錯誤數量: {df_eval['agent_response'].str.contains('❌').sum()} 組")

print("\n🔍 前 8 筆預覽:")
display(df_eval.head(8))




📥 讀取對話資料: gs://willy-poc-crm-agent/ai-eng-test-sample-conversations.json
✅ 載入 323 組對話


🤖 處理對話: 100%|██████████| 323/323 [06:37<00:00,  1.23s/it]

✅ 已儲存: /home/jupyter/jtcg_outputs/jtcg_agent_runs.csv

📊 Intent 分布:
detected_intent
FAQ        209
PRODUCT     43
HANDOFF     38
ORDER       33
Name: count, dtype: int64

📈 總共處理: 323 組對話
   有效查詢: 323 組
   錯誤數量: 0 組

🔍 前 8 筆預覽:


,idx,user_query,detected_intent,agent_response,source_links,is_in_scope_and_correct
0,1,請問你們的退換貨政策是什麼？,FAQ,JTCG Shop 提供 7 天鑑賞期（含例假日）。商品需保持全新、完整包裝與配件。若為非瑕...,,
1,2,保固多久？維修怎麼申請？,FAQ,請備妥訂單編號與故障影片/照片，於線上表單提交 RMA。客服會於工作日內回覆檢測與處理方式。,,
2,3,發票可以開三聯式嗎？要怎麼填統編？,FAQ,本店提供電子發票，出貨完成後將以 Email 寄送或於會員中心可下載查詢。若需三聯式發票，請...,,
3,4,出貨大概幾天？有免運嗎？,FAQ,"現貨商品於 1–2 個工作天出貨；預購商品依頁面標示為準。標準運送滿 NT$1,500 免運...",,
4,5,VESA 100x100 的螢幕可以用哪支臂？,PRODUCT,根據您的需求，以下是幾款可能適合的商品：\n\n1. 【JTCG 壁掛式伸縮臂（至 34 吋...,https://example.com/jtcg/p/JTCG-WALL-ARM-34 ht...,
5,6,我的桌板厚 85mm，夾具能裝嗎？,FAQ,您好！關於您的桌板厚度問題，我們的夾具建議安裝於 10–85mm 厚的桌板。因此，您的 85...,,
6,7,我要找真人客服,HANDOFF,已了解你需要真人協助，請提供 Email（我們會先驗證格式再轉接）。,,
7,8,轉真人，謝謝,HANDOFF,已了解你需要真人協助，請提供 Email（我們會先驗證格式再轉接）。,,


In [16]:
# 可選：上傳回 GCS
upload_to_gcs = True
if upload_to_gcs:
    gcs_output = f"gs://{BUCKET}/outputs/jtcg_agent_runs.csv"
    u = urlparse(gcs_output)
    blob = gcs.bucket(u.netloc).blob(u.path.lstrip("/"))
    blob.upload_from_filename(local_csv)
    print(f"☁️  已上傳至: {gcs_output}")

☁️  已上傳至: gs://willy-poc-crm-agent/outputs/jtcg_agent_runs.csv


In [15]:
# === Debug Cell: 直接檢查 search_knowledge 回傳結果 ===

def debug_search_knowledge(query_text: str, top_k: int = 5):
    print(f"\n===== query: {query_text} =====")
    rows = list(search_knowledge(query_text, top_k=top_k))
    if not rows:
        print("（沒有任何結果）")
        return
    for i, r in enumerate(rows, start=1):
        print(f"\n--- hit #{i} ---")
        print("id      :", getattr(r, "id", None))
        print("title   :", (getattr(r, "title", "") or "").strip())
        print("answer  :", (getattr(r, "answer", "") or "").strip())
        print("url     :", (getattr(r, "url", "") or "").strip())
        print("distance:", getattr(r, "distance", None))

# 測試你關心的兩題
debug_search_knowledge("保固多久？維修怎麼申請？", top_k=5)
debug_search_knowledge("發票可以開三聯式嗎？要怎麼填統編？", top_k=5)



===== query: 保固多久？維修怎麼申請？ =====

--- hit #1 ---
id      : FAQ-RMA-019
title   : 保固申請與維修流程
answer  : 請備妥訂單編號與故障影片/照片，於線上表單提交 RMA。客服會於工作日內回覆檢測與處理方式。
url     : 
distance: 0.5687414929442566

--- hit #2 ---
id      : FAQ-WAR-002
title   : 保固與維修
answer  : 臂架與支架類商品享有 1 年有限保固。非人為損壞可申請檢測與維修；需提供訂單編號或發票資訊。
url     : 
distance: 0.588869413627291

--- hit #3 ---
id      : FAQ-REG-038
title   : 保固登錄與是否可轉移
answer  : 部分商品提供線上保固登錄；保固通常限原始購買者，不隨轉售轉移。實際以商品頁或憑證說明為準。
url     : 
distance: 0.7116209913365729

--- hit #4 ---
id      : FAQ-DOA-034
title   : 到貨外箱受損或零件缺失怎麼辦
answer  : 請立即拍攝外箱與內容物狀況，連同訂單編號於 7 日內回報客服。我們會協助更換缺件或安排售後處理。
url     : 
distance: 0.7831296329729255

--- hit #5 ---
id      : FAQ-INT-010
title   : 海外與離島配送
answer  : 目前主要配送台灣本島；離島/外島運費與時程將於結帳頁顯示。海外配送可先透過客服評估可行方案與預估費用。
url     : 
distance: 0.8095635219715992

===== query: 發票可以開三聯式嗎？要怎麼填統編？ =====

--- hit #1 ---
id      : FAQ-INV-003
title   : 發票與開立方式
answer  : 本店提供電子發票，出貨完成後將以 Email 寄送或於會員中心可下載查詢。若需三聯式發票，請於下單時備註統編與抬頭。
url     : 
distance: 0.6